In [12]:
import numpy as np
import networkx as nx
from collections import Counter
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score


def generate_graphs(n0=150, n1=150, seed=42,
                    n_min=20, n_max=40,
                    p_min=0.05, p_max=0.2, m_min=2, m_max=3):
    rand = np.random.default_rng(seed)
    graphs, labels = [], []

    for _ in range(n0):
        n = int(rand.integers(n_min, n_max + 1))
        p = float(rand.uniform(p_min, p_max))
        G = nx.erdos_renyi_graph(n, p)
        graphs.append(G)
        labels.append(0)

    for _ in range(n1):
        n = int(rand.integers(n_min, n_max + 1))
        m = int(rand.integers(m_min, m_max + 1))
        m = max(1, min(m, n - 1))
        G = nx.barabasi_albert_graph(n, m)
        graphs.append(G)
        labels.append(1)

    return graphs, np.array(labels, dtype=int)


def shortest_path_feature_vector(G, max_len, normalize=True):
    spl = dict(nx.all_pairs_shortest_path_length(G))
    lengths = []
    
    for d in spl.values():
        lengths.extend([L for L in d.values() if L > 0]) 

    counts = np.zeros(max_len, dtype=float) 
    for L in lengths:
        if L >= 1 and L <= max_len:
            counts[L - 1] += 1.0
    if normalize and counts.sum() > 0:
        counts /= counts.sum()
    return counts


def shortest_path_kernel(train_graphs, test_graphs):
    all_graphs = list(train_graphs) + list(test_graphs)
    # общий максимум длины кратчайших путей чтобы выровнять размерность векторов
    max_len = 1
    for G in all_graphs:
        lmax = 1
        for d in nx.all_pairs_shortest_path_length(G):
            _, dist_map = d
            if dist_map:
                lmax = max(lmax, max(dist_map.values()))
        max_len = max(max_len, lmax)

    Phi_train = np.vstack([shortest_path_feature_vector(G, max_len, normalize=True) for G in train_graphs])
    K_train = Phi_train @ Phi_train.T

    Phi_test = np.vstack([shortest_path_feature_vector(G, max_len, normalize=True) for G in test_graphs])
    K_test = Phi_test @ Phi_train.T

    return K_train, K_test


def wl_features(graphs, h=3, use_degree_init=True, normalize=True):
    labels_per_graph = []
    for G in graphs:
        if use_degree_init:
            labels = {v: f"d{G.degree(v)}" for v in G.nodes()}
        else:
            labels = {v: "c0" for v in G.nodes()}
        labels_per_graph.append(labels)

    vocab = {}
    def ensure_id(token):
        if token not in vocab:
            vocab[token] = len(vocab)
        return vocab[token]

    feat_counts = [Counter() for _ in graphs]

    #0
    for gi, labels in enumerate(labels_per_graph):
        for lbl in labels.values():
            idx = ensure_id(f"0|{lbl}")
            feat_counts[gi][idx] += 1

    # wl итерации
    current = labels_per_graph
    for it in range(1, h + 1):
        new_all = []
        for gi, G in enumerate(graphs):
            labels = current[gi]
            new_labels = {}
            for v in G.nodes():
                multiset = sorted(labels[nb] for nb in G.neighbors(v))
                new_lbl = f"{labels[v]}|{'_'.join(multiset)}"
                token = f"{it}|{new_lbl}" 
                new_labels[v] = token
                idx = ensure_id(token)
                feat_counts[gi][idx] += 1
            new_all.append(new_labels)
        current = new_all
        
    dim = len(vocab)
    X = np.zeros((len(graphs), dim), dtype=float)
    for i, cnt in enumerate(feat_counts):
        for j, val in cnt.items():
            X[i, j] = val
        if normalize and X[i].sum() > 0:
            X[i] /= X[i].sum()
    return X


def wl_kernel(train_graphs, test_graphs, h=3):
    all_graphs = list(train_graphs) + list(test_graphs)
    X_all = wl_features(all_graphs, h=h, use_degree_init=True, normalize=True)
    n_tr = len(train_graphs)
    X_tr, X_te = X_all[:n_tr], X_all[n_tr:]
    K_train = X_tr @ X_tr.T
    K_test = X_te @ X_tr.T
    return K_train, K_test



def tune_and_fit_svc(K_train, y, C_grid=(0.1, 1, 10, 25, 50), cv_splits=5, seed=0):
    best_C, best_score = None, -np.inf
    skf = StratifiedKFold(n_splits=cv_splits, shuffle=True, random_state=seed)
    for C in C_grid:
        scores = []
        for tr_idx, va_idx in skf.split(np.zeros(len(y)), y):
            K_tr = K_train[np.ix_(tr_idx, tr_idx)]
            K_va = K_train[np.ix_(va_idx, tr_idx)]
            clf = SVC(kernel='precomputed', C=C)
            clf.fit(K_tr, y[tr_idx])
            y_hat = clf.predict(K_va)
            scores.append(accuracy_score(y[va_idx], y_hat))
        mean_acc = float(np.mean(scores))
        if mean_acc > best_score:
            best_score, best_C = mean_acc, C

    clf = SVC(kernel='precomputed', C=best_C)
    clf.fit(K_train, y)
    return clf, best_C, best_score


def evaluate(clf, K_test, y_test):
    y_pred = clf.predict(K_test)
    acc = accuracy_score(y_test, y_pred)
    prec, rec, f1, _ = precision_recall_fscore_support(y_test, y_pred, average='binary', zero_division=0)
    scores = clf.decision_function(K_test)
    auc = roc_auc_score(y_test, scores)
    return dict(accuracy=acc, precision=prec, recall=rec, f1=f1, roc_auc=auc)

# 1)data
graphs, y = generate_graphs(n0=400, n1=400, seed=0)
G_tr, G_te, y_tr, y_te = train_test_split(graphs, y, test_size=0.25, random_state=0, stratify=y)

# 2)shortest path kernel
Ktr_sp, Kte_sp = shortest_path_kernel(G_tr, G_te)
sp_clf, sp_C, sp_cv = tune_and_fit_svc(Ktr_sp, y_tr, C_grid=(0.1, 1, 10, 25, 50), cv_splits=5, seed=0)
sp_metrics = evaluate(sp_clf, Kte_sp, y_te)

print("Shortest-Path Kernel:")
print(f"Best C (CV): {sp_C} | CV accuracy: {sp_cv:.4f}")
for k, v in sp_metrics.items():
    print(f"{k}: {v:.4f}")

# 3)wl kernel
Ktr_wl, Kte_wl = wl_kernel(G_tr, G_te, h=3)
wl_clf, wl_C, wl_cv = tune_and_fit_svc(Ktr_wl, y_tr, C_grid=(0.1, 1, 10, 25, 50), cv_splits=5, seed=0)
wl_metrics = evaluate(wl_clf, Kte_wl, y_te)

print("\nWeisfeiler–Lehman Kernel:")
print(f"Best C (CV): {wl_C} | CV accuracy: {wl_cv:.4f}")
for k, v in wl_metrics.items():
    print(f"{k}: {v:.4f}")


Shortest-Path Kernel:
Best C (CV): 50 | CV accuracy: 0.8083
accuracy: 0.8100
precision: 0.7348
recall: 0.9700
f1: 0.8362
roc_auc: 0.8378

Weisfeiler–Lehman Kernel:
Best C (CV): 50 | CV accuracy: 0.9717
accuracy: 0.9700
precision: 0.9434
recall: 1.0000
f1: 0.9709
roc_auc: 0.9986


In [11]:
N_RUNS = 10
sp_results, wl_results = [], []

for run in range(N_RUNS):
    graphs, y = generate_graphs(n0=400, n1=400, seed=run)
    G_tr, G_te, y_tr, y_te = train_test_split(
        graphs, y, test_size=0.25, random_state=run, stratify=y
    )

    # Shortest-Path Kernel
    Ktr_sp, Kte_sp = shortest_path_kernel(G_tr, G_te)
    sp_clf, sp_C, sp_cv = tune_and_fit_svc(
        Ktr_sp, y_tr, C_grid=(0.1, 1, 10, 25, 50), cv_splits=5, seed=run
    )
    sp_metrics = evaluate(sp_clf, Kte_sp, y_te)
    sp_results.append(sp_metrics)

    # Weisfeiler–Lehman Kernel
    Ktr_wl, Kte_wl = wl_kernel(G_tr, G_te, h=3)
    wl_clf, wl_C, wl_cv = tune_and_fit_svc(
        Ktr_wl, y_tr, C_grid=(0.1, 1, 10, 25, 50), cv_splits=5, seed=run
    )
    wl_metrics = evaluate(wl_clf, Kte_wl, y_te)
    wl_results.append(wl_metrics)

# усредняем
def mean_std(results, key):
    vals = [r[key] for r in results]
    return np.mean(vals), np.std(vals)

print("=== Mean ± Std over 10 runs ===")
for name, results in [("Shortest-Path", sp_results), ("Weisfeiler–Lehman", wl_results)]:
    print(f"\n{name} Kernel:")
    for k in results[0].keys():
        mean, std = mean_std(results, k)
        print(f"{k:10s}: {mean:.4f} ± {std:.4f}")

=== Mean ± Std over 10 runs ===

Shortest-Path Kernel:
accuracy  : 0.8025 ± 0.0285
precision : 0.7330 ± 0.0320
recall    : 0.9560 ± 0.0314
f1        : 0.8291 ± 0.0223
roc_auc   : 0.8603 ± 0.0271

Weisfeiler–Lehman Kernel:
accuracy  : 0.9700 ± 0.0110
precision : 0.9573 ± 0.0134
recall    : 0.9840 ± 0.0111
f1        : 0.9704 ± 0.0107
roc_auc   : 0.9954 ± 0.0044
